## Basic Settings

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

from sssd_cp.core.imputers.SSSDS4Imputer import SSSDS4Imputer
from sssd_cp.core.model_specs import create_forecast_mask
from sssd_cp.data.dataloader import ArDataLoader
from sssd_cp.training.utils import training_loss
from sssd_cp.utils.utils import find_repo_root, sampling

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_workers = 4

## Parameters

In [ ]:
# inference_sample_size
m = 1
# series_length
T = 3

In [ ]:
std = 1
intercept = 0
k = 1
training_num_series = 10000
testing_num_series = 1000
coefficients = [0.8]
season = None

batch_size = 128
epochs = 1000

seeds = list(range(training_num_series + testing_num_series))
model_save_path = f'model_checkpoint_m={m}_T={T}.pth'

## Data Generation
- Generate AR(1) data with $\phi = 0.8$
- Training data size: 10000
- Testing data size: 1000

In [ ]:
data_loader = ArDataLoader(
    coefficients=coefficients,
    training_num_series=training_num_series,
    testing_num_series=testing_num_series,
    series_length=T,
    std=std,
    intercept=intercept,
    season=season,
    batch_size=batch_size,
    device=device,
    num_workers=num_workers,
    seeds=seeds,
)

train_loader = data_loader.train_dataloader
test_loader = data_loader.test_dataloader

## Model Configuration

In [ ]:
from sssd_cp.utils.utils import load_yaml_file, calc_diffusion_hyperparams

current_dir = os.getcwd()
repo_root = find_repo_root(current_dir)

configs_dir = os.path.join(repo_root, "configs")
model_config_path = os.path.join(configs_dir, "model.yaml")
training_config_path = os.path.join(configs_dir, "training.yaml")
inference_config_path = os.path.join(configs_dir, "inference.yaml")

# Load the configuration files
model_config = load_yaml_file(model_config_path)
training_config = load_yaml_file(training_config_path)
inference_config = load_yaml_file(inference_config_path)

diffusion_hyperparams = calc_diffusion_hyperparams(**model_config["diffusion"], device=device)
display(diffusion_hyperparams)

In [ ]:
net = SSSDS4Imputer(**model_config.get("wavenet"), device=device)

net = net.to(device)
net = nn.DataParallel(net)

optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)

## Model Training

In [ ]:
losses = []

epoch_pbar = tqdm(range(epochs), desc="Training Epochs")

for epoch in epoch_pbar:
    epoch_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        mask = create_forecast_mask(batch, unseen_length=k, device=device)
        loss_mask = ~mask.bool()
        batch = batch.permute(0, 2, 1)

        optimizer.zero_grad()
        loss = training_loss(
            model=net,
            loss_function=nn.MSELoss(),
            training_data=(batch, batch, mask, loss_mask),
            diffusion_parameters=diffusion_hyperparams,
            generate_only_missing=training_config.get("only_generate_missing"),
            device=device,
        )
        loss.backward()
        optimizer.step()

        epoch_loss += loss.cpu().detach().numpy() / len(train_loader)

    losses.append(epoch_loss)

    epoch_pbar.set_postfix(loss=f'{epoch_loss:.4f}')


torch.save(net.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")
print(f"Finished training for {len(losses)} epochs.")

In [ ]:
# Plotting the losses
plt.figure(figsize=(10, 6))
plt.plot(range(1, epochs + 1), losses, marker='o', linestyle='-', color='b', label='Training Loss')

# Adding title and labels
plt.title('Training Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.grid(True)
plt.show()

## Inference

In [ ]:
results = []
test_data_list = []
net.eval()

for batch in tqdm(test_loader, desc="Processing Batches"):
    batch = batch.to(device)
    mask = create_forecast_mask(batch, unseen_length=k, device=device)
    batch = batch.permute(0, 2, 1)
    
    generated_samples = sampling(
        net=net,
        size=batch.shape,
        diffusion_hyperparams=diffusion_hyperparams,
        cond=batch,
        mask=mask,
        sample_size=m,
        only_generate_missing=1,
        device=device,
    )
    results.append(generated_samples)
    test_data_list.append(batch)

## Evaluation
1. 1st MSE: $E(\hat{y}_t - y_t)^2$
2. 2nd MSE: $E(\hat{y}_t - \phi \cdot y_{t-1})^2$
3. Bias: $E(\hat{y}_t - \phi \cdot y_{t-1})$
4. Variance: $Var(\hat{y}_t - \phi \cdot y_{t-1})$

In [ ]:
mean_results = np.vstack([np.mean(result, axis=0) for result in results]).squeeze()
test_data = torch.stack(test_data_list).cpu().numpy().squeeze()
test_data.shape

In [ ]:
# Check dimensions of test_data and mean_results
if test_data.ndim == 1 and mean_results.ndim == 1:
    # 1-dimensional case
    mse_metric_1 = mean_squared_error(test_data, mean_results)
    mse_metric_2 = mean_squared_error(test_data * coefficients[0], mean_results)
    bias = np.mean(mean_results - test_data * coefficients[0])
    var = np.var(mean_results - test_data * coefficients[0])
elif test_data.ndim == 2 and mean_results.ndim == 2:
    # 2-dimensional case
    if k < test_data.shape[1]:
        mse_metric_1 = mean_squared_error(test_data[:, -k], mean_results[:, -k])
        mse_metric_2 = mean_squared_error(test_data[:, -(k+1)] * coefficients[0], mean_results[:, -k])
        bias = np.mean(mean_results[:, -k] - test_data[:, -(k+1)] * coefficients[0])
        var = np.var(mean_results[:, -k] - test_data[:, -(k+1)] * coefficients[0])
    else:
        raise IndexError(f"k = {k} is out of bounds for array with shape {test_data.shape}")
else:
    raise ValueError("Inconsistent dimensions between test_data and mean_results")

In [ ]:
metrics = {
    '1st MSE': [mse_metric_1],
    '2nd MSE': [mse_metric_2],
    'Bias': [bias],
    'Variance': [var]
}

display(pd.DataFrame(metrics))